**Text generation:**
 
This problem also has been solved in Mathematica, which runs faster here. Although the method used to generate the Markov chain and is quite similar.  

Part I&II.
Here we model the Markov chain model to generate a character sequence with a set of states given by (k-gram s). 
We start with the initial current state which is a 3-gram selected randomly form the chosen corpus (wiki pages here). The next character (new state) will be selected randomly from the subset of k-grams that matches the last two characters of the current state. 
The probability of the transition to a new state is given by the frequency of the matched subset k-grams in the selected corpus. 
we build a dictionary data structure with {(key1,key2): [values]} where the keys are two consecutive characters (key1,key2): with the values define the list of all characters that follows the key on the text (or two consecutive characters in the case of build_dictII(...) method). 

Duplicates are allowed in the values since the transition probabilities are given by frequency of values. 

The algorithm starts with choosing a random {(key1,key2): value[i]} (3-gram tuple) from the text, which construct the start of the sentence. The last two characters of the tuple is our new currant state {(key1,key2):[value]} and we pick a random value form the value list and add it to the sentence. We iterate until we reach the desired length. So we are interested in the transition given by the pattern
([ch, i],[ch, i+1],[ch, i+3])->([ch, i+1],[ch, i+2],[ch, i+4]) for the character sets {[ch, i]}. 

Part III:
finding the probability of generating a given text with this Markovia model:
For Further explanation regarding finding the probability of generating a given text with this Markovia model see the Mathematica Notebook. 

In [366]:
'''Pulling texts from wikipedia pages'''
import os
import time
import urllib2
from bs4 import BeautifulSoup
from urllib2 import Request
import urllib
import re 

"""download and parse the Philosophy wiki pages and all the links therein,
    Since downloading all the content takes few minutes you can only load 
    the file wiki.txt which has all the wiki data """
def check_wiki():
    
    try:
        link = "https://en.wikipedia.org/wiki/Philosophy"
        resp = urllib2.urlopen(link)
        soup = BeautifulSoup(resp)
        
    except Exception as e:
        print(str(e))
    
    text = [''.join(s.findAll(text=True)).encode('utf8') for s in soup.findAll('p')]
    text=' '.join(text)
    text_phyl = text
    #print(len(text))
    #print (text[1])
    #text=re.sub(r'\\+x\d\d..|x\\',"",str(text))
    #text=re.sub(r'.x\\.',"",text)
    #text=re.sub(r'x[^a-zA-Z]',"",text)
    #print ("NEWwwwwwwwwww\n\n")
    #print (text)
    
    links=soup.find_all("a")
    link_list=[]
    for link in links:
        item=str(link.get('href'))
        s=item.find("/wiki/")
        if s>-1:
            link_list.append(item)
    
    for item in link_list[:]:
        try:
            link = "http://en.wikipedia.org"+item.lower()
            print link
            resp = urllib2.urlopen(link)
            soup = BeautifulSoup(resp)
            text_l = [''.join(s.findAll(text=True)).encode('utf8') for s in soup.findAll('p')]
            text_l=' '.join(text_l)
            text=text + text_l
            
        except Exception as e:
            print(str(e))
    
    return text



In [367]:
text = check_wiki() 

http://en.wikipedia.org/wiki/wikipedia:protection_policy#semi
http://en.wikipedia.org/wiki/philosophy_(disambiguation)
http://en.wikipedia.org/wiki/category:philosophy
http://en.wikipedia.org/wiki/file:philbar_3.png
http://en.wikipedia.org/wiki/plato
http://en.wikipedia.org/wiki/immanuel_kant
HTTP Error 404: Not Found
http://en.wikipedia.org/wiki/friedrich_nietzsche
http://en.wikipedia.org/wiki/buddha
http://en.wikipedia.org/wiki/confucius
http://en.wikipedia.org/wiki/averroes
http://en.wikipedia.org/wiki/category:philosophers
http://en.wikipedia.org/wiki/list_of_aestheticians
http://en.wikipedia.org/wiki/list_of_epistemologists
http://en.wikipedia.org/wiki/list_of_ethicists
http://en.wikipedia.org/wiki/list_of_logicians
http://en.wikipedia.org/wiki/list_of_metaphysicians
http://en.wikipedia.org/wiki/list_of_social_and_political_philosophers
http://en.wikipedia.org/wiki/category:philosophical_traditions
http://en.wikipedia.org/wiki/african_philosophy
http://en.wikipedia.org/wiki/analyt

In [374]:
"""If do not want to wait for text = check_wiki() skip it text data is in wiki.txt"""
import os
import time
import urllib2
from bs4 import BeautifulSoup
from urllib2 import Request
import urllib
import re 

# to save the wiki texts in a text file uncomment this part
#with open("wiki.txt", "w") as text_file:
#    text_file.write(text)

In [ ]:
"""If skipped text = check_wiki() run this line to load the wiki.txt"""

In [4]:
text=""
with open("wiki.txt", "r") as text_file:
    text=text_file.read()

In [7]:
from random import choice
from __future__ import division
from decimal import Decimal
from decimal import *
import os
import time
import urllib2
from bs4 import BeautifulSoup
from urllib2 import Request
import urllib
import re 
getcontext().prec = 48

"""we build a dictionary data structure with {(key1,key2): [values]} where the keys are
two consecutive characters (key1,key2): with the values defined as the list of all characters 
that follows the key on the text (or two consecutive characters in the 
case of build_dictII(...) ). Duplicates are allowed in the values since 
the transition probabilities are given by the frequency of values. The dictionary data structures makes the the keys look up fast. 
"""

def build_dict(words):

    dic = {}
    alphabet_len=0
    chain_le=3
    for i in range(len(words)-chain_le+1):
        try:
            w1, w2, w3 = words[i], words[i+1], words[i+2]
        except Exception as e:
            print(str(e))
            #break
        key = (w1, w2)
        if key not in dic:
            dic[key] = []
        dic[key].append(w3)
         
    return dic


"""markov_gen_char(dd,gen_len): generate a Markovian chain
with lenght=gen_len and the input dictionary=dd based on a
given corpus.
The method starts with choosing a 
random {(key1,key2): value[i]} (3-gram tuple) 
from the text, which construct the start of the sentence. 
The last two characters of the tuple is our new currant state
{(key1,key2):[value]} and we pick a random value form the 
value list and add it to the sentence. 
We iterate until we reach the desired length.
transition given by the pattern (c1,c2,c3)->(c2,c3,c4) for the character sets {ci}. """
##gen_len= lenght of the desired sentence
##dd = dictionary built based on the wiki data {(key1,key2): value[i]}

def markov_gen_char(dd,gen_len = 200):

    sent = []
    key_list = [key for key in dd.keys()]
    key_pick = choice(key_list)
    w1, w2 = key_pick
    sent.append(w1)
    sent.append(w2)
    #print(key_pick)
    for _ in range(gen_len-2):
        try:
            #print(dd[key_pick],key_pick)

            w3=choice(dd[key_pick])
            sent.append(w3)

        except Exception as e:
            print(str(e))

        key_pick = (w2 , w3)
        w1 , w2 = key_pick
    return ''.join(sent)

In [8]:
"""generating the random text with 200 character"""
chars = list(text)
dd= build_dict(chars)
markov_gen_char(dd,200)


'[x eal-fan the of Enly be the ace phiclumarge sidgenclued cal any wit on Poethe bas asionslationsin and Iralits Midepical inal-Kumateduch Karxism artaim withe as tion to dic orist ch alumaytion all Go'

In [17]:
markov_gen_char(dd,200)


'07, Che Natee-etiould Klebal lose a hossars whilempotat prophaved al Kined.[169] The Refork an) and Tufferemsers, sua.[27902] Hux, por thetwerred psed, terand tre In the 178" aboults, an phiplescras c'

In [370]:
##
##build_dict_II(words): build a dictionary based on the wiki text for  
## consecutive (key):(value) pairs: {(key1,key2): (value1,value2)}
##
##  

def build_dict_II(words):

    dic = {}
    alphabet_len=0
    chain_le=3
    for i in range(len(words)-chain_le):
        try:
            w1, w2, w3, w4 = words[i], words[i+1], words[i+2], words[i+3]
        except Exception as e:
            print(str(e))
            #break
        key = (w1, w2)
        if key not in dic:
            dic[key] = []
        dic[key].append((w3,w4))
         
    return dic

Finding the probability of generating a given text with this Markovia model:
For Further explanation regarding formalism to find the probability of generating a given text with this Markovia model see the Mathematica Notebook. (see the implementation below) 

In [372]:
#finding the probability of generating a given target text from the trained markovian 
#text generator

chars = list(text)
dd= build_dict(chars)
dd2=build_dict_II(chars)

target ='Machine learning is a scientific discipline that explores the construction and study of algorithms that can learn form data. [1] Such algorithms operate by building a model from example inputs and using that to make predictions or decisions, [2] :2 rather than following strictly static program instructions. Machine learning is closely related to and often overlaps with computational statistics; a discipline which also specializes in prediction-making.'

chain_le=3
prob=[]
tprob=1
dic=dd
dic2=dd2



for i in range(len(target)-chain_le):
#for i in range(5):

    try:
        w1, w2, w3, w4 = target[i], target[i+1], target[i+2], target[i+3] 
    except Exception as e:
        print(str(e))
        #break
    alphabet_len=len(chars)-2
    key = (w1, w2)
    n_ch1=1
    if key in dic and i==1:
        if w3 in dic[key]:
            n_ch1=dic[key].count(w3)
        else:
            alphabet_len+=len(dic[key])
    
        prob_ch1=1.0*(n_ch1/alphabet_len)
        prob.append(prob_ch1)
        print(n_ch1,prob_ch1)
    n_ch_i=1
    n_ch_j=1
    if key in dic2 and i>1:
        n_ch_j=dic2[key].count((w3,w4))+1
        
    key_n = (w3, w4)
    if key_n in dic2 and i>1:
        n_ch_i=len(dic2[key_n])+1
    prob.append(1.0*(n_ch_j/n_ch_i))
    #print("oioi",n_ch_i,n_ch_j)
    #tprob=1.0*(n_ch_j/n_ch_i)*tprob

        

(10823, 0.00043366366379722605)


In [373]:
tprob=1
for x in prob:
    tprob = Decimal(x)*Decimal(tprob)
print tprob

3.17143436485866335646114227260785015710028683258E-686


Experiment: We can use words instead of characters to generate random sentences that are
less gibirish!! 

In [191]:
def markov_gen_word(dd,gen_len = 10):

    sent = []
    key_list = [key for key in dd.keys()]
    key_pick = choice(key_list)
    w1, w2 = key_pick
    sent.append(w1)
    sent.append(w2)
    #print(key_pick)
    for _ in range(gen_len-2):
        try:
            w3=choice(dd[key_pick])
            sent.append(w3)

        except Exception as e:
            print(str(e))

        key_pick = (w2 , w3)
        w1 , w2 = key_pick
    return ' '.join(sent)


In [89]:
text=text.lower()
words = text.split()
dd=build_dict(words)
markov_gen_word(dd,200)

"texts. the renaissance () period saw the development of tantra and iranian-islamic influences. buddhism mostly disappeared from india after the disappearance of buddhism from india, these philosophical traditions developed complex phenomenological psychologies termed 'abhidharma'. mahayana philosophers such as the geonim of the humanities, is also the malest (and the whitest). while other areas of the serious philosophical investigation have multiplied. not only is the emphasis on reasoning as employed in the biomedical and life in a circle so that a statement is included in its entirety and ontology, the study of the various buddhist schools of islamic philosophy developed under the influence of mao zedong, while a chinese pragmatism under hshih and new confucianism's rise was influenced by the greeks included metaphysics (with competing theories such tao, yin and yang, ren and li which, along with chinese buddhism, directly influenced korean philosophy, vietnamese philosophy and of 